In [ ]:
import numpy as np
import pandas as pd

from pathlib import Path
from matplotlib import pyplot as plt

In [ ]:
from dotenv import load_dotenv
from pathlib import Path
import os

load_dotenv()

output_path = Path(os.environ.get("OUTPUT_DIR"))
figures_path = Path(os.environ.get("FIGURES_DIR"))
wave_folder = Path(os.environ.get("WAVE_BENCHMARK_DIR"))
wave_folder

In [ ]:
target_hdf_advection = "1D_Advection_Sols_beta0.4.hdf5"
target_hdf_darcy = "2D_DarcyFlow_beta1.0.hdf5"

In [ ]:
advection_path = output_path / "1d_advection"
advection_fv_path = advection_path / "fv" / "results"
advection_col_path = advection_path / "col" / "results"

darcy_path = output_path / "2d_darcy"
darcy_fv_path = darcy_path / "fv" / "results"
darcy_col_path = darcy_path / "col" / "results"

In [ ]:
import yaml

def get_dfs(results_path, hdf_path):
    args_paths = results_path.glob("**/args.yaml")
    N_pdes = []
    dfs = []
    for p in args_paths:
        with open(p, "r") as f:
            args = yaml.safe_load(f)
            if not hdf_path in args["hdf"]:
                continue
            if "Advection" in hdf_path:
                N_pde = args["N_pde_t"] * args["N_pde_x"]
            elif "Darcy" in hdf_path:
                N_pde = args["N_pde_xy"]**2
            else:
                raise ValueError("Unknown problem")
            N_bc = args["N_bc"]
            N_pdes.append(N_pde)
        csv_path = p.with_name("results.csv")
        df = pd.read_csv(csv_path)
        dfs.append(df)
    order = np.argsort(N_pdes)
    N_pdes = [N_pdes[i] for i in order]
    dfs = [dfs[i] for i in order]
    return N_pdes, dfs

In [ ]:
N_pdes_advection_fv, dfs_advection_fv = get_dfs(advection_fv_path, target_hdf_advection)
N_pdes_advection_col, dfs_advection_col = get_dfs(advection_col_path, target_hdf_advection)

N_pdes_darcy_fv, dfs_darcy_fv = get_dfs(darcy_fv_path, target_hdf_darcy)
N_pdes_darcy_col, dfs_darcy_col = get_dfs(darcy_col_path, target_hdf_darcy)

In [ ]:
def get_L_infs(dfs):
    return [df["L_inf_err"].mean() for df in dfs]

def get_RMSEs(dfs):
    return [np.sqrt(df["MSE"]).mean() for df in dfs]

L_infs_advection_fv = get_L_infs(dfs_advection_fv)
L_infs_advection_col = get_L_infs(dfs_advection_col)

RMSEs_advection_fv = get_RMSEs(dfs_advection_fv)
RMSEs_advection_col = get_RMSEs(dfs_advection_col)

L_infs_darcy_fv = get_L_infs(dfs_darcy_fv)
L_infs_darcy_col = get_L_infs(dfs_darcy_col)

RMSEs_darcy_fv = get_RMSEs(dfs_darcy_fv)
RMSEs_darcy_col = get_RMSEs(dfs_darcy_col)

In [ ]:
wave_df_fv = pd.read_csv(wave_folder / "errs_fv.csv")
wave_df_col = pd.read_csv(wave_folder / "errs_col.csv")

wave_df_fv = wave_df_fv.drop(wave_df_fv.columns[0], axis='columns')
wave_df_col = wave_df_col.drop(wave_df_col.columns[0], axis='columns')
max_depth = 5
num_dimensions = 3

levels = np.arange(-1, max_depth+1)
num_obs = lambda level: 0 if level == -1 else (2**num_dimensions) ** level
num_obs = np.vectorize(num_obs)

fv_vals = [float(x) for x in wave_df_fv.values.flatten()]
col_vals = [float(x) for x in wave_df_col.values.flatten()]
# Construct points for scatter plot
tile_factor = int(len(fv_vals) / len(levels))
points_fv = [np.tile(num_obs(levels), tile_factor), fv_vals]
points_col = [np.tile(num_obs(levels), tile_factor), col_vals]

fv_means = list(wave_df_fv.mean())
fv_stds = list(wave_df_col.std())

col_means = list(wave_df_col.mean())
col_stds = list(wave_df_col.std())

fv_errs = np.array(fv_stds)
col_errs = np.array(col_stds)

In [ ]:
from tueplots import bundles

plt.rcParams.update(bundles.neurips2024(nrows=1, ncols=2))
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Set1.colors)

fig, ax = plt.subplots(1, 3)

# Advection
ax[0].plot(N_pdes_advection_fv, L_infs_advection_fv, ".-", label="GP-FVM")
ax[0].plot(N_pdes_advection_col, L_infs_advection_col, ".-", label="Collocation")
ax[0].set_xscale("log", base=4)
ax[0].set_yscale("log", base=10)
ax[0].set_xlabel('$N_{\mathrm{PDE}}$')
ax[0].set_ylabel("Maximum absolute error")
ax[0].axhline(y=2e-1, linestyle="--", color=plt.cm.Set1.colors[2])
ax[0].axhline(y=1.7, linestyle="--", color=plt.cm.Set1.colors[6])
ax[0].text(4**4, 2.3e-1, "FNO", color=plt.cm.Set1.colors[2], ha='center')
ax[0].text(4**(5), 1.25, "PINN, U-Net", color=plt.cm.Set1.colors[6], ha='center')

# ax[0].legend()
ax[0].set_title("\\textbf{a)} 1D Advection ($\\beta=0.4$)")

# Darcy
ax[1].plot(N_pdes_darcy_fv, L_infs_darcy_fv, ".-", label="GP-FVM")
ax[1].plot(N_pdes_darcy_col, L_infs_darcy_col, ".-", label="Collocation")
ax[1].set_xscale("log", base=4)
ax[1].set_yscale("log", base=10)
ax[1].set_xlabel('$N_{\mathrm{PDE}}$')
ax[1].axhline(y=1.1e-1, linestyle="--", color=plt.cm.Set1.colors[2])
ax[1].axhline(y=9e-2, linestyle="--", color=plt.cm.Set1.colors[3])
ax[1].set_title("\\textbf{b)} 2D Darcy Flow ($\\beta=1.0$)")
ax[1].text(4**3, 1.2e-1, "FNO", color=plt.cm.Set1.colors[2], ha='center')
ax[1].text(4**3, 9.5e-2, "U-Net", color=plt.cm.Set1.colors[3], ha='center')
# ax[1].legend()

# Wave
ax[2].plot(num_obs(levels), fv_means, ".-", label="GP-FVM")
ax[2].plot(num_obs(levels), col_means, ".-", label="Collocation")

# Scatter plot
ax[2].scatter(*points_fv, marker=".", color="C0", alpha=0.4)
ax[2].scatter(*points_col, marker=".", color="C1", alpha=0.4)

ax[2].set_xscale('symlog', base=(2**num_dimensions))
ax[2].set_yscale('log')
ax[2].set_xlabel('$N_{\mathrm{PDE}}$')
ax[2].set_title("\\textbf{c)} 2D Wave Equation")

fig.savefig(f"{figures_path}/advection_darcy_benchmark.pdf")